In [ ]:
# Função para calcular a ETo:

def calc_eto(df_estacao, alt, lat):   
    """
    :df_estacao: dataframe com os "dados brutos" ou "corrigidos e pré-processados".
    :alt::altitude.
    :lat:latitude.
    :Retorna um novo DataFrame com todos os parâmetros de ETo calculados.
    """

    # Inserir a coluna 'Tmed' e calcular:
    df_estacao.insert(5, 'Tmed', '', allow_duplicates=False)
    df_estacao['Tmed'] = (df_estacao['Tmax'] + df_estacao['Tmin'])/2

    # Inserir a coluna 'URmed' e calcular:
    df_estacao.insert(8, 'Urmed', '', allow_duplicates=False)
    df_estacao['Urmed'] = (df_estacao['Urmax'] + df_estacao['Urmin'])/2

    # Formatar a data:
    df_estacao['Data'] = pd.to_datetime(df_estacao['Data'], format='%Y-%m-%d')

    # Inserir a coluna 'dia_ano' e calcular:
    df_estacao.insert(1,'Dia_Ano','',allow_duplicates=False)
    df_estacao['Dia_Ano'] = df_estacao.iloc[:]['Data'].dt.dayofyear

    # Inserir a coluna de anos e meses:
    df_estacao['Anos'] = df_estacao['Data'].dt.year
    df_estacao['Meses'] = df_estacao['Data'].dt.month

    # Renomear os meses:
    df_estacao.loc[df_estacao.Meses == 1,'Meses']='Jan'
    df_estacao.loc[df_estacao.Meses == 2,'Meses']='Fev'
    df_estacao.loc[df_estacao.Meses == 3,'Meses']='Mar'
    df_estacao.loc[df_estacao.Meses == 4,'Meses']='Abr'
    df_estacao.loc[df_estacao.Meses == 5,'Meses']='Mai'
    df_estacao.loc[df_estacao.Meses == 6,'Meses']='Jun'
    df_estacao.loc[df_estacao.Meses == 7,'Meses']='Jul'
    df_estacao.loc[df_estacao.Meses == 8,'Meses']='Ago'
    df_estacao.loc[df_estacao.Meses == 9,'Meses']='Set'
    df_estacao.loc[df_estacao.Meses == 10,'Meses']='Out'
    df_estacao.loc[df_estacao.Meses == 11,'Meses']='Nov'
    df_estacao.loc[df_estacao.Meses == 12,'Meses']='Dez'

    #################################################################################################
    # Calcular: "y", "es", "ea" e "Delta":

    # Constante psicrométrica (gamma):
    df_estacao['P'] = 101.3*np.power(((293-0.0065*alt)/293), 5.26)
    df_estacao['gamma'] = df_estacao['P']*0.665*np.float_power(10, -3)

    # es mínimo, máximo e médio:
    df_estacao['es_tmin'] = 0.6108*np.exp(17.27*df_estacao['Tmin']/(df_estacao['Tmin']+237.3))
    df_estacao['es_tmax'] = 0.6108*np.exp(17.27*df_estacao['Tmax']/(df_estacao['Tmax']+237.3))
    df_estacao['es_tmed'] = 0.6108*np.exp(17.27*df_estacao['Tmed']/(df_estacao['Tmed']+237.3))

    # Pressão de saturação de vapor (es):
    df_estacao['es'] = (df_estacao['es_tmax'] + df_estacao['es_tmin'])/2

    # Pressão atual de vapor (ea):
    df_estacao['ea'] = (df_estacao['es'] * df_estacao['Urmed'])/100

    # Declividade da curva de pressão de vapor em relação à temperatura (DELTA):
    df_estacao['Delta'] = (4098 * df_estacao['es_tmed'])/np.float_power(df_estacao['Tmed'] + 237.3, 2)
    #################################################################################################
    # Calcular a radiação líquida
    '''
    Passos:
    1º) Ra -> Radiação solar no topo da atmosfera
    2º) Rso -> Radiação solar incidente na ausência de nuvens
    3º) Rnl -> Saldo de radiação de ondas longas
    4º) Rns -> Saldo de radiação de ondas curtas
    5º) Rn -> Saldo de radiação
    '''
    ######################################################
    # 1º) Ra -> Radiação solar no topo da atmosfera
    ######################################################

    # Calcular o dr:
    df_estacao['dr'] = 1 + 0.033*np.cos(2*np.pi*df_estacao['Dia_Ano']/365)

    # Calcular o delta:
    df_estacao['delta'] = 0.409*np.sin((2*np.pi*df_estacao['Dia_Ano']/365) - 1.39)

    # Calcular o phi:
    df_estacao['phi'] = lat*np.pi/180

    # Calcular o ômega:
    df_estacao['omega_s'] = np.arccos(-1*np.tan(df_estacao['phi'])*np.tan(df_estacao['delta']))

    # Constante é igual a 24*60*G_sc/pi
    const = 24*60*0.0820/np.pi

    # Por fim, aplicar na equação de Ra:
    df_estacao['Ra']=const*df_estacao['dr']*(df_estacao['omega_s']*np.sin(df_estacao['phi'])*np.sin(df_estacao['delta'])+np.cos(df_estacao['phi'])*np.cos(df_estacao['delta'])*np.sin(df_estacao['omega_s']))

    ############################################################
    # 2º) Rso -> Radiação solar incidente na ausência de nuvens
    ############################################################
    df_estacao['Rso'] = (0.75 + (2*alt*0.00001)) * df_estacao['Ra']

    ############################################################
    # 3º) Rnl -> Saldo de radiação de ondas longas
    ############################################################
    sigma = 4.903*np.float_power(10, -9)
    df_estacao['a'] = (np.float_power(df_estacao['Tmax']+273.16, 4) + np.float_power(df_estacao['Tmin']+273.16, 4))/2
    df_estacao['b'] = 0.34-0.14*np.sqrt(df_estacao['ea'])
    df_estacao['c'] = 1.35*(df_estacao['Rad']/df_estacao['Rso']) - 0.35
    df_estacao['Rnl'] = sigma * df_estacao['a'] * df_estacao['b'] * df_estacao['c']

    ############################################################
    # 4º) Rns -> Saldo de radiação de ondas curtas
    ############################################################
    alpha = 0.23
    df_estacao['Rns'] = (1-alpha) * df_estacao['Rad']

    ############################################################
    # 5º) Rn -> Saldo de radiação ###
    ############################################################
    df_estacao['Rn'] = df_estacao['Rns'] - df_estacao['Rnl']    

    #################################################################################################
    # Calcular a Evapotransporação de Referência (ETo)
    df_estacao['ETo']=(0.408*df_estacao['Delta']*df_estacao['Rn']+df_estacao['gamma']*(37/(df_estacao['Tmed']+273))*df_estacao['u']*(df_estacao['es']-df_estacao['ea']))/(df_estacao['Delta']+df_estacao['gamma']*(1+0.34*df_estacao['Urmed']))

    return df_estacao